# Projet Chatbot
## Emilie Longuevre / Léna Nock / Valentine Carteron

### 1. Importation des packages

In [32]:
import requests
from bs4 import BeautifulSoup, Tag, NavigableString
from urllib.error import HTTPError, URLError
from urllib.request import urlopen
import pickle
import pandas as pd
import re

In [2]:
#site surlequel on réalise le webscrapping: Foire aux questions de la page Vinted
site = "https://www.vinted.fr/help"
# commande pour accéder aux données 
response = requests.get(site)

### 2. Récupérer toutes les urls des catégories du centre d'aide 
On récupère les urls des neuf catégories du centre d'aide.

In [3]:
urls_categorie1 = []
href = (BeautifulSoup(response.text)
          .select("html body div.site div.container div.faq div.faq__categories a.faq__category"))
for ref in href :
    urls_categorie1.append("https://www.vinted.fr" + ref.get('href'))
urls_categorie1

['https://www.vinted.fr/help/16-comment-ca-marche',
 'https://www.vinted.fr/help/4-vendre',
 'https://www.vinted.fr/help/5-acheter',
 'https://www.vinted.fr/help/313-envoi',
 'https://www.vinted.fr/help/418-paiements-et-transferts',
 'https://www.vinted.fr/help/15-confiance-et-securite',
 'https://www.vinted.fr/help/392-mon-compte-parametres',
 'https://www.vinted.fr/help/383-communaute',
 'https://www.vinted.fr/help/103-se-connecter']

On enregistre cette liste au format .txt pour conserver une trace de ces urls.

In [6]:
# with open('urls_categorie1.txt', 'w') as f:
#     for url in urls_categorie1:
#         f.write(f'{url}\n')

# urls_categorie1 = open("urls_categorie1.txt", "r")
# urls_categorie1 = urls_categorie1.read().split("\n")
# urls_categorie1 = urls_categorie1[:-1]
# urls_categorie1

['https://www.vinted.fr/help/16-comment-ca-marche',
 'https://www.vinted.fr/help/4-vendre',
 'https://www.vinted.fr/help/5-acheter',
 'https://www.vinted.fr/help/313-envoi',
 'https://www.vinted.fr/help/418-paiements-et-transferts',
 'https://www.vinted.fr/help/15-confiance-et-securite',
 'https://www.vinted.fr/help/392-mon-compte-parametres',
 'https://www.vinted.fr/help/383-communaute',
 'https://www.vinted.fr/help/103-se-connecter']

Pour accéder aux réponses on doit passer par des sous-catégories de question. On va donc récupérer l'ensemble des urls menant aux réponses. On créé donc une fonction afin d'optimiser le code. Au maximum, on a trois sous catégories, donc on lance la fonction 'récupération_url' trois fois.

In [4]:
def recuperation_url(urls) :
    urls_categorie_sup = []
    for url in urls : 
        # on précise l'url qu'on va explorer
        response = requests.get(url)
        # on sélectionne les liens présents dans ces pages
        href = (BeautifulSoup(response.text)
                  .select("html body div.site div.container div.faq a.c-cell__link"))
        # s'il y a une sous catégorie:
        if href != [] :
            for ref in href :
                # on ajoute l'url qui suit dans la liste des catégories supérieurs
                urls_categorie_sup.append("https://www.vinted.fr" + ref.get('href'))
        # s'il n'y a pas de sous catégories:
        else : 
            # on ajoute l'url actuel
            urls_categorie_sup.append(url)
    return urls_categorie_sup

In [5]:
urls_categorie2 = recuperation_url(urls_categorie1)

In [9]:
len(urls_categorie2)

52

In [15]:
# with open('urls_categorie2.txt', 'w') as f:
#     for url in urls_categorie2:
#         f.write(f'{url}\n')

# urls_categorie2 = open("urls_categorie2.txt", "r")
# urls_categorie2 = urls_categorie2.read().split("\n")
# urls_categorie2 = urls_categorie2[:-1]

In [110]:
urls_categorie3 = recuperation_url(urls_categorie2)
len(urls_categorie3)

216

In [28]:
# with open('urls_categorie3.txt', 'w') as f:
#     for url in urls_categorie3:
#         f.write(f'{url}\n')

# urls_categorie3 = open("urls_categorie3.txt", "r")
# urls_categorie3 = urls_categorie3.read().split("\n")
# urls_categorie3 = urls_categorie3[:-1]

In [111]:
urls_categorie4 = recuperation_url(urls_categorie3)
len(urls_categorie4)

221

In [29]:
# with open('urls_categorie4.txt', 'w') as f:
#     for url in urls_categorie4:
#         f.write(f'{url}\n')

# urls_categorie4 = open("urls_categorie4.txt", "r")
# urls_categorie4 = urls_categorie4.read().split("\n")
# urls_categorie4 = urls_categorie4[:-1]

In [26]:
urls_categorie4

['https://www.vinted.fr/help/26-vendre-etape-par-etape',
 'https://www.vinted.fr/help/25-acheter-sur-vinted-toutes-les-etapes',
 'https://www.vinted.fr/help/373-vendre-sur-vinted-c-est-gratuit',
 'https://www.vinted.fr/help/356-comment-partager-mon-retour-d-experience-sur-vinted',
 'https://www.vinted.fr/help/265-comment-utiliser-le-centre-d-aide',
 'https://www.vinted.fr/help/375-ajouter-un-article-etape-par-etape',
 'https://www.vinted.fr/help/26-vendre-etape-par-etape',
 'https://www.vinted.fr/help/373-vendre-sur-vinted-c-est-gratuit',
 'https://www.vinted.fr/help/52-qu-est-ce-que-je-peux-vendre-sur-vinted',
 'https://www.vinted.fr/help/220-comment-echanger-un-article',
 'https://www.vinted.fr/help/375-ajouter-un-article-etape-par-etape',
 'https://www.vinted.fr/help/601-quelles-photos-dois-je-mettre-en-ligne-pour-mes-articles-de-marque',
 'https://www.vinted.fr/help/48-comment-prendre-de-bonnes-photos',
 'https://www.vinted.fr/help/49-comment-bien-decrire-mon-article',
 'https://ww

### 3. Récupérer les réponses associés aux urls des questions

In [10]:
def ValidTag(monTag):
    if type(monTag) == Tag and monTag.has_attr("class"):
        # Tag 'tab-content' : parfois il y avait plusieurs onglets dans les réponses (format web, format ios et format android) pour afficher la réponse selon le device que l'utilisateur utilise
            # nous avons choisi de prendre uniquement le web (à voir) => donc pour l'instant on retourne False 
        # Tag 'nav-tabs-horizontal' : correspond à la barre qui affiche si on veut regarder la réponde pour IOS/ANDROID ou WEB
        # Tag 'table-wrapper' supprimé : on supprime le texte présent dans ce Tag qui correspond à des tableaux afin de remplacer par une meilleure mise en page de tableau
        if ' '.join(monTag.attrs["class"]) in ['tab-content', 'nav-tabs-horizontal', '“table-wrapper”']:
            return False
    return True

In [11]:
def getSelectedText(monTag):
    texte = []
    for child in monTag.children :
        if type(child) == NavigableString :
            if (child.string).strip() != '' and (child.string).strip() != ' ' :
                texte.append( " " + (child.string).strip().replace('\n', '') )
                
        ## Récupération des tableaux
        if type(child) == Tag and child.has_attr("class") and ' '.join(child.attrs["class"]) in ['“table-wrapper”']:
            tables = child.findAll('table')


            l = []
            nbr_valeur_ligne = []
            colonne = []
        
            # pour chaque ligne du tableau:
            for i,tr in enumerate(tables[0].findAll('tr')):
                # si la ligne contient un 'rowspan' c'est à dire une case pour plusieurs lignes
                if 'rowspan' in str(tr) : 
                    # récupère le numéro de cette ligne
                    valeur_depart = i 
                    # on ajoute à une liste l'ensemble des textes présents sur la ligne
                l.append(tr.text.replace('\n', '|#|'))
            # on récupère l'entête du tableau
            liste_entete = l[0].split('|#|')
            # on supprime les élément vides
            while '' in liste_entete : 
                liste_entete.remove('')
                
            a_change = []
            # on parcourt les lignes du tableaux ayant l'attribut 'td'
            for i,td in enumerate(tables[0].findAll('td')) :
                # si cet attribut contient un 'rowspan':
                if td.has_attr("rowspan") :
                    # on récupère le numéro de la colonne correspondant
                    num_colonne = i % len(liste_entete) 
                    # nombre de lignes concernées par cette exception
                    nbr_fois_lig = td.attrs['rowspan']
                    # valeur correspondante qui doit être recopiée
                    valeur_a_ajouter = td.text
                    # on récupère l'indice des lignes ayant une valeur manquante
                    a_change = [valeur_depart + o for o in range(2,int(nbr_fois_lig)+1)]

                    
            # création d'un compteur correspondant aux index des lignes        
            n = 1 
            # pour chaque ligne du tableau (sans le nom des colonnes)
            for mot in l[1::] : 
                n +=1
                # on découpe la ligne selon '|#|'
                # on obtient une liste contenant l'ensemble des mots de la ligne
                ligne = mot.split('|#|') 
                while '' in ligne : 
                    ligne.remove('')
                # si l'index de la ligne est présent dans la liste à changer
                if n in a_change: 
                    # on ajoute la valeur à ajouter au numéro de colonne correspondant
                    ligne[num_colonne:num_colonne] = [valeur_a_ajouter]
                # affichage final du tableau
                for i in range(0,len(liste_entete)) : 
                    texte.append( (str(liste_entete[i]) +  " : " + str(ligne[i])).replace('\n', '') )
                texte.append( '------' )      
                
        
                
        if type(child) == Tag and ValidTag(child) and child.name != 'table':
            if getSelectedText(child).strip() != '' and getSelectedText(child).strip() != ' ' :
                texte.append( getSelectedText(child).strip().replace('\n', '') )
                
                
        if type(child) == Tag and child.name == 'table':
            tables = child.find('table')
        
            l = []
            nbr_valeur_ligne = []
            colonne = []
        
            # pour chaque ligne du tableau:
            for i,tr in enumerate([child][0].findAll('tr')):
                # si la ligne contient un 'rowspan' c'est à dire une case pour plusieurs lignes
                if 'rowspan' in str(tr) : 
                    # récupère le numéro de cette ligne
                    valeur_depart = i 
                    # on ajoute à une liste l'ensemble des textes présents sur la ligne
                l.append(tr.text.replace('\n', '|#|'))
            # on récupère l'entête du tableau
            liste_entete = l[0].split('|#|')
            # on supprime les élément vides
            while '' in liste_entete : 
                liste_entete.remove('')
                
            a_change = []
            # on parcourt les lignes du tableaux ayant l'attribut 'td'
            for i,td in enumerate([child][0].findAll('td')) :
                # si cet attribut contient un 'rowspan':
                if td.has_attr("rowspan") :
                    # on récupère le numéro de la colonne correspondant
                    num_colonne = i % len(liste_entete) 
                    # nombre de lignes concernées par cette exception
                    nbr_fois_lig = td.attrs['rowspan']
                    # valeur correspondante qui doit être recopiée
                    valeur_a_ajouter = td.text
                    # on récupère l'indice des lignes ayant une valeur manquante
                    a_change = [valeur_depart + o for o in range(2,int(nbr_fois_lig)+1)]

                    
            # création d'un compteur correspondant aux index des lignes        
            n = 1 
            # pour chaque ligne du tableau (sans le nom des colonnes)
            for mot in l[1::] : 
                n +=1
                # on découpe la ligne selon '|#|'
                # on obtient une liste contenant l'ensemble des mots de la ligne
                ligne = mot.split('|#|') 
                while '' in ligne : 
                    ligne.remove('')
                # si l'index de la ligne est présent dans la liste à changer
                if n in a_change: 
                    # on ajoute la valeur à ajouter au numéro de colonne correspondant
                    ligne[num_colonne:num_colonne] = [valeur_a_ajouter]
                # affichage final du tableau
                for i in range(0,len(liste_entete)) : 
                    texte.append( (str(liste_entete[i]) +  " : " + str(ligne[i])).replace('\n', '') )
                texte.append( '------' )             
        
    texte_final = '\n'.join(texte)
    return texte_final

On peut donc désormais récupérer l'ensemble des textes qui correspondent aux réponses.

In [12]:
liste_reponses = []
for url in urls_categorie4 :
    try :
        html = urlopen(url)
    except (HTTPError, URLError) as e :
        sys.exit(e)
    
    # On récupère les réponses dans la variable 'liste_reponses'
    # On remplace certains codes par leur signification
    bsObj = BeautifulSoup(html, "lxml")
    maDiv = bsObj.find("div", class_="site").find("div", class_="body-content").find("div", class_="faq__question-answer faq-content")
    texte = getSelectedText(maDiv).strip().replace('\xa0', ' ').replace('\u200b', ' ').replace('\u2192', ' ').replace('\u2194', ' ').replace('\u2028', ' ')
    liste_reponses.append(texte)
liste_reponses[0:3]

['Vendre, étape par étape\nVendre un article, c’est simple et gratuit ! Voilà comment faire :\nMettre en ligne un article.\nAjoute debelles photos de l’article, écris unedescription détaillée,fixe le prix de ton article, choisis unetaille pour le colis .La mise en ligne d’un article est gratuite.Mettre en ligne plus d’un article aide à vendre plus vite.\nConfigure ton profil.\nParamètre tonporte-monnaie Vinted et faisvérifier ton compte .Sélectionne le transport et les modes de paiement dans tes paramètres. Tu peux également faire une promotion sur ton catalogue.En ajoutant unedescription et une photo sur ton profil , tu renforceras la confiance des autres membres.\nAttends les acheteurs.\nSois prêt à répondre aux questions des acheteurs.Clique sur le boutonFaire une offre si tu veux proposer un meilleur prix.Tu seras notifié durant tout le processus d’achat de ton acheteur.\nArticle vendu : il est temps de l’envoyer.\nEmballe ton article (une touche personnelle est toujours appréciée)

In [14]:
# with open('Reponses.txt', 'w') as f:
#     for texte in liste_reponses:
#         f.write(f'{str(texte)}|#|')


In [18]:
# ajouter le retour à la ligne

### 4. Récupérer toutes les questions

In [15]:
# On récupère le nom des neuf premières catégories 
question = (BeautifulSoup(response.text)
          .select("html body div.site div.container div.faq div.faq__categories  div.faq__category-title"))

liste_categories = []
for i in range(0,len(question)) :
    liste_categories.append(question[i].text.strip())
liste_categories

['Comment ça marche',
 'Vendre',
 'Acheter',
 'Envoi',
 'Paiements et transferts',
 'Confiance et sécurité',
 'Mon compte & Paramètres',
 'Communauté',
 'Se connecter']

On créé une fonction permettant de récupérer l'ensemble des chemins menant aux questions.

In [16]:
def chemin_question(urls_cat, liste_prec):
    liste_chemin = []
    num=0
    # pour chaque url de la catégorie:
    for url in urls_cat : 
            response = requests.get(url)
            # on récupère le titre des sous catégories/questions
            question = (BeautifulSoup(response.text)
                  .select("html body div.site div.container div.faq div.c-cell__title"))
            # s'il y a des questions:
            if question != [] :
                # pour chaque numéro de question:
                for i in range(0,len(question)):
                    # on ajoute le chemin précédent + le titre de la sous catégorie/question
                    liste_chemin.append(liste_prec[num] + ' ' + question[i].text.strip())
            else:
                # sinon on garde le chemin actuel
                liste_chemin.append( liste_prec[num] )
            num+=1        
    return liste_chemin

In [17]:
chemin1 = chemin_question(urls_categorie1, liste_categories)

In [18]:
chemin2 = chemin_question(urls_categorie2, chemin1)

In [30]:
chemin3 = chemin_question(urls_categorie3, chemin2)

In [33]:
liste_questions = chemin_question(urls_categorie4, chemin3)

### 5. Création de catégories thématiques de questions :

In [78]:
liste_categories_them = []
for num in range(0,len(chemin1)) :
    for elem in range(0,len(liste_questions)) :
        if re.findall(chemin1[num], str(liste_questions[elem])) != [] :
            liste_categories_them.append(chemin1[num])
liste_categories_them[0:15]

['Comment ça marche Vendre, étape par étape',
 'Comment ça marche Acheter sur Vinted : toutes les étapes',
 "Comment ça marche Vendre sur Vinted, c'est gratuit ?",
 'Comment ça marche Comment partager mon retour d’expérience sur Vinted ?',
 "Comment ça marche Comment utiliser le Centre d'aide ?",
 'Vendre Vendre sur Vinted',
 'Vendre Vendre sur Vinted',
 'Vendre Vendre sur Vinted',
 'Vendre Vendre sur Vinted',
 'Vendre Vendre sur Vinted',
 'Vendre Télécharger un article',
 'Vendre Télécharger un article',
 'Vendre Télécharger un article',
 'Vendre Télécharger un article',
 'Vendre Télécharger un article']

### 6. Création d'un dataframe contenant les questions et les réponses 

In [79]:
dataQR = pd.DataFrame({'question': liste_questions, 'reponse': liste_reponses, 'thematiques': liste_categories_them})
dataQR['question'] = dataQR['question'].astype(str)
dataQR['thematiques'] = dataQR['thematiques'].astype(str)
dataQR.head(10)

,question,reponse,thematiques
0,"Comment ça marche Vendre, étape par étape","Vendre, étape par étape\nVendre un article, c’...","Comment ça marche Vendre, étape par étape"
1,Comment ça marche Acheter sur Vinted : toutes ...,Acheter sur Vinted : toutes les étapes\nTrouve...,Comment ça marche Acheter sur Vinted : toutes ...
2,"Comment ça marche Vendre sur Vinted, c'est gra...","Vendre sur Vinted, c'est gratuit ?\nIl n'y a p...","Comment ça marche Vendre sur Vinted, c'est gra..."
3,Comment ça marche Comment partager mon retour ...,Comment partager mon retour d’expérience sur V...,Comment ça marche Comment partager mon retour ...
4,Comment ça marche Comment utiliser le Centre d...,Comment utiliser le Centre d'aide ?\nSur notre...,Comment ça marche Comment utiliser le Centre d...
5,Vendre Vendre sur Vinted Ajouter un article ét...,Ajouter un article étape par étape\nVoici comm...,Vendre Vendre sur Vinted
6,"Vendre Vendre sur Vinted Vendre, étape par étape","Vendre, étape par étape\nVendre un article, c’...",Vendre Vendre sur Vinted
7,"Vendre Vendre sur Vinted Vendre sur Vinted, c'...","Vendre sur Vinted, c'est gratuit ?\nIl n'y a p...",Vendre Vendre sur Vinted
8,Vendre Vendre sur Vinted Qu'est-ce que je peux...,Qu'est-ce que je peux vendre sur Vinted ?\nVoi...,Vendre Vendre sur Vinted
9,Vendre Vendre sur Vinted Comment échanger un a...,Comment échanger un article\nÉchanger des arti...,Vendre Vendre sur Vinted


In [80]:
# On vérifie combien de questions et de réponses on a.
print("Le nombre de questions est : ", len(dataQR['question']))
print("Le nombre de réponses est : ", len(dataQR['reponse']))

# afficher les 15 premières données
dataQR['question'].values[:15] 

Le nombre de questions est :  221
Le nombre de réponses est :  221


array(['Comment ça marche Vendre, étape par étape',
       'Comment ça marche Acheter sur Vinted : toutes les étapes',
       "Comment ça marche Vendre sur Vinted, c'est gratuit ?",
       'Comment ça marche Comment partager mon retour d’expérience sur Vinted ?',
       "Comment ça marche Comment utiliser le Centre d'aide ?",
       'Vendre Vendre sur Vinted Ajouter un article étape par étape',
       'Vendre Vendre sur Vinted Vendre, étape par étape',
       "Vendre Vendre sur Vinted Vendre sur Vinted, c'est gratuit ?",
       "Vendre Vendre sur Vinted Qu'est-ce que je peux vendre sur Vinted ?",
       'Vendre Vendre sur Vinted Comment échanger un article',
       'Vendre Télécharger un article Ajouter un article étape par étape',
       'Vendre Télécharger un article Quelles photos dois-je mettre en ligne pour mes articles de marque ?',
       'Vendre Télécharger un article Comment prendre de bonnes photos ?',
       'Vendre Télécharger un article Comment bien décrire mon article ?',

On enregistre le dataframe obtenu ici pour pouvoir l'utiliser dans un nouveau notebook.

In [81]:
dataQR.to_csv("dataQR.csv", sep=';', index = False)